In [1]:
import os
import sys
sys.path.insert(0,'..')

from bidst.transformers import StatAggregator

In [2]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   └── test_pipeline
│       └── steps
│           ├── skullstripping
│           │   └── sub-02
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       ├── sub-02_ses-retest_T1w.nii.gz
│           │       │       └── sub-02_ses-retest_T1w_brain.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-02_ses-test_T1w_brain.nii.gz
│           └── tissuesegmentation
│               └── sub-02
│                   ├── ses-retest
│                   │   └── anat
│                   │       ├── sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_class-GM_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_mixeltype.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_pve

In [3]:
import nibabel as nib
from functools import reduce
from operator import mul

def get_tissue_stats(subject_folder):
    
    ext_dict = {}
    ext_dict['WM_volume'] = 'WM_probtissue'
    ext_dict['GM_volume'] = 'GM_probtissue'
    ext_dict['CSF_volume'] = 'CSF_probtissue'
    
    files_list = os.listdir(subject_folder)
    
    results = {}
    for key in ext_dict.keys():
        cur_file = [item for item in files_list if ext_dict[key] in item]
        if len(cur_file) > 1:
            raise ValueError('Several candiadates in the folder')
        else:
            cur_file = cur_file[0]
        cur_img = nib.load(subject_folder + '/' + cur_file)
        voxel_sum = cur_img.get_data()
        voxel_sum = (voxel_sum >= 0.5).sum()
        voxel_vol = reduce(mul, cur_img.header.get_zooms())
        all_vol = voxel_sum * voxel_vol
        results[key] = all_vol
    
    return results

In [4]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline0_ds114'
gather_steps = ('tissuesegmentation', dict(extensions='WM_probtissue.nii.gz'))

IDS = ['02']

transformer = StatAggregator(function=get_tissue_stats,
    project_path=project_path,
                                        pipeline_name=pipeline_name,
                                        gather_steps=gather_steps)
transformer.fit_transform(IDS)

SUBJECT 02
{'extensions': 'WM_probtissue.nii.gz'}
[File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat'), File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-test/anat/sub-02_ses-test_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='test', type='probtissue', modality='anat')]
in_file File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat')
session retest
path /work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-retest/anat
in_file File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissueseg

CSF_volume      GM_volume      WM_volume
02 retest  247122.684898  449870.986028  468183.542938
   test    235231.048205  411737.814129  440105.344166

In [5]:
from bids.grabbids import BIDSLayout

layout = BIDSLayout('/work/bidst/tests/data/ds114/')

In [6]:
layout.get(extensions = 'WM_probtissue.nii.gz', subject = '02')

[File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat'),
 File(filename='/work/bidst/tests/data/ds114/derivatives/test_pipeline/steps/tissuesegmentation/sub-02/ses-test/anat/sub-02_ses-test_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='test', type='probtissue', modality='anat')]

In [7]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   └── test_pipeline
│       └── steps
│           ├── skullstripping
│           │   └── sub-02
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       ├── sub-02_ses-retest_T1w.nii.gz
│           │       │       └── sub-02_ses-retest_T1w_brain.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-02_ses-test_T1w_brain.nii.gz
│           └── tissuesegmentation
│               └── sub-02
│                   ├── ses-retest
│                   │   └── anat
│                   │       ├── sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_class-GM_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_mixeltype.nii.gz
│                   │       ├── sub-02_ses-retest_T1w_brain_pve